In [1]:
import numpy as np
import torch
import torch.nn as nn 
import torch.nn.functional as F
import zipfile    
import os
import pickle

from torch.utils.data import TensorDataset, DataLoader

In [105]:
pauli_operators = [np.array([[1,0],[0,1]]), np.array([[0,1],[1,0]]), np.array([[0,-1j],[1j,0]]), np.array([[1,0],[0,-1]]) ]
inital_states = 1/2*np.array([ (pauli_operators[0]+pauli_operators[1]), (pauli_operators[0]-pauli_operators[1]), \
                               (pauli_operators[0]+pauli_operators[2]), (pauli_operators[0]-pauli_operators[2]),\
                               (pauli_operators[0]+pauli_operators[3]), (pauli_operators[0]-pauli_operators[3]) ])

pauli_operators = torch.tensor(pauli_operators)
inital_states   = torch.tensor(inital_states)     

nInput  = 12 # L=4, 3*4=12
nOutput = 18 # 3*6 =18


In [31]:
f = open("simu_data_Gaussian_L=4.ds", 'rb')
data = pickle.load(f)
f.close()

In [140]:
inputs  = data["training_inputs"]
targets = data["training_targets"]

In [149]:
np.array(inputs).shape, np.array(targets).shape

((1000, 4, 3), (1000, 3, 6))

In [161]:
torch.from_numpy(np.array(inputs)).shape, torch.from_numpy(np.array(targets)).shape

(torch.Size([1000, 4, 3]), torch.Size([1000, 3, 6]))

In [166]:
torch.reshape(torch.from_numpy(np.array(targets)), (-1,18)).shape

torch.Size([1000, 18])

In [44]:
TensorDataset(torch.from_numpy(np.array(inputs)), torch.from_numpy(np.array(targets)))

In [62]:
torch.Tensor(inputs).shape

torch.Size([1000, 4, 3])

In [66]:
torch.reshape(torch.Tensor(inputs), (-1,12)).shape

torch.Size([1000, 12])

In [167]:
class QubitNNmodel(nn.Module):
    """
    the NN-based model to train the qubit
    """
    def __init__(self, inputs, targets, nInput=12, nHidden=3, nOutput=18) -> None:
        """
        inputs  : list of ctrl_params, size = L*3
        targets : list of toggling-frame msmt_results, size=18
        """
        super().__init__()
        self.inputs  = inputs       # x in ML = ctrl_params ;
        self.targets = targets      # y in ML = msmt_results; 
        #nInput  = 12                # len(x) in ML      
        self.batch_size = 10 
        self.train_ds = TensorDataset(torch.from_numpy(np.array(inputs)), \
                                      torch.from_numpy(np.array(targets)))                 # Define dataset
        self.train_dl = DataLoader(self.train_ds, self.batch_size, shuffle=False)          # Define data loader
        ### /////// how to split data into training & testing set ??  torch.utils.data.random_split(dataset, [0.75, 0.25])
        self.layers   = nn.Sequential(
            nn.Linear(nInput, nHidden),         # 1st hidden layer
            nn.Sigmoid(),
            nn.Linear(nHidden,nHidden),         # 2nd hidden layer
            nn.Sigmoid(), 
            nn.Linear(nHidden,nHidden),         # 3rd hidden layer    
            nn.Sigmoid(),
            nn.Linear(nHidden, nHidden),        # 4th hidden layer    
            nn.Sigmoid(),
            nn.Linear(nHidden, nOutput),        # Output layer    
        )                                                                                    # Make the structured-NN       
        self.model    = self.layers(torch.reshape(torch.Tensor(inputs), (-1,nInput)))        # Define the model
        #self.opt      = torch.optim.SGD(self.model.parameters(), lr=1e-3)                    # Define optimizer
        self.loss_fn  = nn.MSELoss()                                                         # Define loss function
        self.losses   = []                                                                   # List of losses in training
    def opt(self):
        return torch.optim.SGD(self.model.parameters(), lr=1e-3)
    
    def forward(self,x):
        x = torch.reshape(torch.Tensor(x), (-1,nInput))
        return (self.layers(x))
    def fit(self, ePochs):
        """
        define a utility function to train the model
        """
        for epoch in ePochs:
            for x, y in self.train_dl:
                y_Pred = self.model(x)                # Generate predictions
                loss = self.loss_fn(y_Pred,y)         # measure the loss
                #Perform gradient descent:
                self.loss_fn.backward()
                self.opt.step()
                self.opt.zero_grad()
            #print('Training loss: ', self.loss_fn(self.model(self.x), self.y))
            if (epoch + 1) % 1000 == 0:
                print('Loss after epoch %5d: %.3f' %(epoch + 1, loss))
            # keep track of losses
            self.losses.append(loss.detach().numpy)            
    

In [169]:
model_ = QubitNNmodel( inputs , targets)
#model_.forward(inputs)

In [170]:
class MyNN(nn.Module):
    """
    define NN model
    """
    def __init__(self, nInput=12, nHidden=3, nOutput=18) -> None:
        super().__init__()
        self.layers   = nn.Sequential(
            nn.Linear(nInput, nHidden),         # 1st hidden layer
            nn.Sigmoid(),
            nn.Linear(nHidden,nHidden),         # 2nd hidden layer
            nn.Sigmoid(), 
            nn.Linear(nHidden,nHidden),         # 3rd hidden layer    
            nn.Sigmoid(),
            nn.Linear(nHidden, nHidden),        # 4th hidden layer    
            nn.Sigmoid(),
            nn.Linear(nHidden, nOutput),        # Output layer    
        )
    def forward(self,x):
        return (self.layers(x))
#model  = MyNN()
#model    

In [150]:
train_ds = TensorDataset(torch.from_numpy(np.array(inputs,dtype='float32')), \
                         torch.from_numpy(np.array(targets,dtype='float32')))                 # Define dataset
train_dl = DataLoader(train_ds, batch_size=5, shuffle=False)
for x, y in train_dl:
    print(y)       

tensor([[[ 0.9983, -0.9983, -0.0349,  0.0349, -0.0307,  0.0307],
         [ 0.0352, -0.0352,  0.9963, -0.9963,  0.0112, -0.0112],
         [ 0.0327, -0.0327, -0.0124,  0.0124,  0.9971, -0.9971]],

        [[ 0.9934, -0.9934, -0.0227,  0.0227,  0.0420, -0.0420],
         [ 0.0127, -0.0127,  0.9950, -0.9950,  0.0119, -0.0119],
         [-0.0447,  0.0447, -0.0084,  0.0084,  0.9894, -0.9894]],

        [[ 0.9804, -0.9804,  0.0604, -0.0604,  0.0553, -0.0553],
         [-0.0608,  0.0608,  0.9827, -0.9827,  0.0070, -0.0070],
         [-0.0556,  0.0556, -0.0176,  0.0176,  0.9975, -0.9975]],

        [[ 0.9834, -0.9834,  0.0570, -0.0570, -0.1074,  0.1074],
         [-0.0632,  0.0632,  0.9922, -0.9922, -0.0136,  0.0136],
         [ 0.1019, -0.1019,  0.0292, -0.0292,  0.9889, -0.9889]],

        [[ 0.9860, -0.9860,  0.0680, -0.0680, -0.0076,  0.0076],
         [-0.0678,  0.0678,  0.9867, -0.9867, -0.0129,  0.0129],
         [ 0.0060, -0.0060,  0.0074, -0.0074,  0.9991, -0.9991]]])
tensor([[[ 9.95

In [184]:
class Training(MyNN):
    def __init__(self, inputs, targets) -> None:
        super().__init__()
        self.inputs = inputs
        self.targets= targets
        self.model    = MyNN()
        self.opt      = torch.optim.SGD(self.model.parameters(), lr=1e-3)
        self.loss_fn  = nn.MSELoss()
        self.losses   = []
        self.Epochs   = 1000
        self.batch_size = 10 
        self.train_ds = TensorDataset(torch.reshape(torch.from_numpy(np.array(inputs ,dtype='float32')), (-1,12)), \
                                      torch.reshape(torch.from_numpy(np.array(targets,dtype='float32')), (-1,18)))                # Define dataset
        self.train_dl = DataLoader(self.train_ds, self.batch_size, shuffle=False)                          # Define data loader
                
    def fit(self):
        for epoch in range(self.Epochs):
            for x, y in self.train_dl:
                #print(x.size, y.size)
                #print(x,y)
                pred = self.model(x)
                loss = self.loss_fn(pred, y)
                loss.backward()
                self.opt.step()
                self.opt.zero_grad()
            if (epoch + 1) % 100 == 0:
                print('Loss after epoch %5d: %.3f' %(epoch + 1, loss))
            # keep track of losses
        

In [185]:
my_training = Training(inputs, targets)

In [186]:
my_training.model

MyNN(
  (layers): Sequential(
    (0): Linear(in_features=12, out_features=3, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=3, out_features=3, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=3, out_features=3, bias=True)
    (5): Sigmoid()
    (6): Linear(in_features=3, out_features=3, bias=True)
    (7): Sigmoid()
    (8): Linear(in_features=3, out_features=18, bias=True)
  )
)

In [187]:
my_training.train_ds

In [188]:
my_training.fit()

Loss after epoch   100: 0.007
Loss after epoch   200: 0.001
Loss after epoch   300: 0.001
Loss after epoch   400: 0.001
Loss after epoch   500: 0.001
Loss after epoch   600: 0.001
Loss after epoch   700: 0.001
Loss after epoch   800: 0.001
Loss after epoch   900: 0.001
Loss after epoch  1000: 0.001
